***
__GEEExp class__
The 'GEEExp' class hosts methods to export the image collections obtained from the GEECol.getcollection method.
- __exportimages__:            exports the separate images to a local directory
- exportimagestack:        exports the images stacked as bands in a multiband image to a local directory
- exportimagestodrive:     exports the separate images to the google drive
- __exportimagestacktodrive__: exports the images stacked as bands in a multiband image to the google drive


for local download exportimages is preferred, for google drive exportimagestacktodrive is preferred
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geeexport

import os
import numpy
import osgeo.gdal
import matplotlib.pyplot
import uuid


__example: different products__

In [ ]:
#
# reference
#
refgeecol            = geeproduct.GEECol_s2scl()
refroipixelsdiameter = 128
#
# parameters
#
eepoint              = geeutils.bobspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'month')
verbose              = False
szoutputdir          = r"C:\tmp"
szfilenameprefix     = "tmp_" + str(uuid.uuid4().hex) + "_"
#
# targets
#
targets = [
    [geeproduct.GEECol_s2ndvi(),              refroipixelsdiameter*2],
    [geeproduct.GEECol_s2fapar(),             refroipixelsdiameter*2],
    [geeproduct.GEECol_s2scl(),               refroipixelsdiameter],
    [geeproduct.GEECol_s2sclconvmask(),       refroipixelsdiameter], # "convolve" takes a while
    [geeproduct.GEECol_s2sclstaticsmask(),    refroipixelsdiameter], # mind you - normally we do this over a year
    [geeproduct.GEECol_s2sclclassfractions(), refroipixelsdiameter], # same here
    [geeproduct.GEECol_s2cloudlessmask(),     refroipixelsdiameter], # experimental (based on gee tutorial)
    [geeproduct.GEECol_s1sigma0('VV', 'ASC'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1sigma0('VH', 'ASC'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1sigma0('VV', 'DES'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1sigma0('VH', 'DES'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1gamma0('VV', 'ASC'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1gamma0('VH', 'ASC'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1gamma0('VV', 'DES'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1gamma0('VH', 'DES'), refroipixelsdiameter*2],
    [geeproduct.GEECol_s1rvi('ASC'),          refroipixelsdiameter*2], # experimental - just for fun
    [geeproduct.GEECol_s1rvi('DES'),          refroipixelsdiameter*2], # same here
    [geeproduct.GEECol_pv333ndvi(),           int(refroipixelsdiameter*20/333) + 2],
    [geeproduct.GEECol_pv333simplemask(),     int(refroipixelsdiameter*20/333) + 2]
]

#
# export and read back first exported file
#
cols = 4
rows = int(len(targets)/cols)
rows = rows if rows*cols == len(targets) else rows + 1
matplotlib.rcParams['figure.figsize'] = [5*cols, 5*rows] # width x height in inches
gridspec = matplotlib.gridspec.GridSpec(rows, cols)
gridspec.update(top = .90, left = 0.10, bottom = 0.05, right = .99, wspace=0.01, hspace=0.2, )
for iIdx, target in enumerate(targets):

    geecol               = target[0]
    roipixelsdiameter    = target[1]
    geecollection = geecol.getcollection(
        eedatefrom, eedatetill, eepoint, roipixelsdiameter, 
        refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose)
    
    szdescription    = geecollection.get('gee_description').getInfo()
    eecenterpoint    = ee.Geometry(geecollection.get('gee_centerpoint'))
    eefirsttrgimage  = geecollection.first()
    xenv = geeutils.squarerasterboundsroi(eecenterpoint, roipixelsdiameter*5, eefirsttrgimage)
    xmmx = eefirsttrgimage.clip(xenv).reduceRegion(ee.Reducer.minMax()).values().getInfo()
    xmin = min(xmmx)
    xmax = max(xmmx)
    #
    # actual export
    #
    geeexport.GEEExp().exportimages(geecollection, szoutputdir, szfilenameprefix=szfilenameprefix, verbose=verbose)
    #
    # read exported file
    #
    szfilenames      = [szfile for szfile in os.listdir(szoutputdir) if szfile.startswith(szfilenameprefix+szdescription)]
    szfile           = os.path.join(szoutputdir, szfilenames[0])
    gdaldataset      = osgeo.gdal.Open(szfile)
    numpyarray       = gdaldataset.ReadAsArray()

    row = int(iIdx/cols)
    col = iIdx - row * cols
    axis = matplotlib.pyplot.subplot(gridspec[row, col])
    axis.set_title(szdescription)
    axis.imshow(numpyarray, cmap='gray', vmin=xmin, vmax=xmax)
gdaldataset = None


__example: timeseries__

In [ ]:
import os
import numpy
import osgeo.gdal
import matplotlib.pyplot
import uuid

#
# reference
#
refgeecol            = geeproduct.GEECol_s2scl()
refroipixelsdiameter = 256
#
# parameters
#
eepoint              = geeutils.bobspoint
eedatefrom           = geeutils.fleecycloudsday
eedatetill           = eedatefrom.advance(1, 'month')
verbose              = False
szoutputdir          = r"C:\tmp"
szfilenameprefix     = "tmp_" + str(uuid.uuid4().hex) + "_"
#
# export s2ndvi and s2sclconvmask
#
geeexport.GEEExp().exportimages(
    geeproduct.GEECol_s2ndvi().getcollection(
        eedatefrom, eedatetill, eepoint, refroipixelsdiameter*2, 
        refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose),
    szoutputdir, szfilenameprefix=szfilenameprefix, verbose=verbose)
geeexport.GEEExp().exportimages(
    geeproduct.GEECol_s2sclconvmask().getcollection(
        eedatefrom, eedatetill, eepoint, refroipixelsdiameter*1, 
        refcollection=refgeecol, refroipixelsdiameter=refroipixelsdiameter, verbose=verbose),
    szoutputdir, szfilenameprefix=szfilenameprefix, verbose=verbose)

ndvifilenames = [str(szfile) for szfile in os.listdir(szoutputdir) if szfile.startswith(szfilenameprefix+"S2ndvi")]
maskfilenames = [str(szfile) for szfile in os.listdir(szoutputdir) if szfile.startswith(szfilenameprefix+"S2sclconvmask")]
ndvifilenames.sort()
maskfilenames.sort()
#print(ndvifilenames)
#print(maskfilenames)
countfiles = len(ndvifilenames)
if (countfiles != len(maskfilenames)) : raise Exception("eeeuuuh...")
#
#
#
cols = 2
rows = countfiles

matplotlib.rcParams['figure.figsize'] = [12, 5*rows] # width x height in inches
gridspec = matplotlib.gridspec.GridSpec(rows, cols)
gridspec.update(top = .90, left = 0.10, bottom = 0.05, right = .99, wspace=0.01, hspace=0.1, )

for iIdx in range(countfiles):
    ndvigdaldataset      = osgeo.gdal.Open(os.path.join(szoutputdir, ndvifilenames[iIdx]))
    ndvinumpyarray       = ndvigdaldataset.ReadAsArray()
    maskgdaldataset      = osgeo.gdal.Open(os.path.join(szoutputdir, maskfilenames[iIdx]))
    masknumpyarray       = maskgdaldataset.ReadAsArray()
    axis = matplotlib.pyplot.subplot(gridspec[iIdx, 0])
    axis.imshow(ndvinumpyarray, cmap='gray', vmin=-1, vmax=1)
    axis = matplotlib.pyplot.subplot(gridspec[iIdx, 1])
    axis.imshow(masknumpyarray, cmap='gray', vmin=0,  vmax=1)
ndvigdaldataset = None
maskgdaldataset = None

    